This is going to be the Jupyter Notebook for the IBM Dataset

In [ ]:
import os
import tarfile
import urllib.request
import pandas as pd

In [ ]:
load_ibm_data = pd.read_csv("C:/Users/alt98/Desktop/Machine Learning Project/Employee_Retention/Dataset - IBM.csv")

In [ ]:
ibm_data = load_ibm_data.copy()
#make sure it works
ibm_data.columns

In [ ]:
#change titles to lowercase
ibm_data.columns = ibm_data.columns.str.lower()

In [ ]:
#let's see what kind of data we have here
ibm_data.info()
# o null values in the dataset, so we can proceed with the analysis

In [ ]:
# Get value counts for all object-type columns
object_columns = ibm_data.select_dtypes(include='object')  # Select object-type columns
value_counts = {col: object_columns[col].value_counts() for col in object_columns.columns}

# Print the value counts for each column
for col, counts in value_counts.items():
    print(f"Value counts for column '{col}':")
    print(counts)
    print()

In [ ]:
ibm_data.describe()


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
ibm_data.hist(bins=50, figsize=(20, 15))
plt.show()